# Foucault'sches Pendel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden in drei Abschnitten verschiedene Aspekte des Foucault'schen Pendels betrachtet. Zunächst beschränken wir uns auf den Fall kleiner Auslenkungen, der durch die dimensionslosen Bewegungsgleichungen 

\begin{align}
 \ddot x &= - x - 2 \omega_\text{E} v_y\\
 \ddot y &= - y + 2 \omega_\text{E} v_x
\end{align}

beschrieben wird, wobei $\omega_\text{E}$ die Rotationsfrequenz der Erde relativ zur Pendelfrequenz angibt. Im ersten Abschnitt wird die Projektion der Pendelmasse auf die $x$-$y$-Ebene numerisch bestimmt,
wobei die Möglichkeit gegeben wird, verschiedene Löser zu verwenden und deren Effizienz zu beurteilen. Im zweiten Abschnitt werden dann die Punkte maximaler Auslenkung bestimmt, um die Drehung der Schwingungsebene numerisch zu erfassen.

Da die Bewegung für kleine Auslenkungen auch analytisch berechnet werden kann, erfolgt im dritten Abschnitt die Verallgemeinerung auf beliebige Auslenkungen. Die dimensionslosen Bewegungsgleichungen lauten dann

\begin{align}
 \ddot\theta &= \frac{1}{2} \dot{\phi}^2 \sin(2\theta) - \sin(\theta)
- \omega_E \dot{\phi} \left[ 2 \sin^2(\theta)\sin(\phi)\cos(\varphi)-\sin(2\theta)\sin(\varphi) \right]\\
 \ddot\phi &= \frac{2 \omega_E \dot{\theta} \sin(\theta) \sin(\phi) \cos(\varphi)-2\cos(\theta) \left(\dot{\phi}+\omega_E \sin(\varphi)\right)\dot{\theta}}{\sin(\theta)}\,.
\end{align}

Der Winkel $\varphi$ gibt hier die geographische Breite an, bei der das Pendel aufgestellt ist.

## Bahnkurve für kleine Auslenkungen

### Importanweisungen

In [ ]:
from math import cos, pi, sin, tan
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Definition der Benutzerschnittstelle

Um eine Kopplung der Parametereinstellungen in verschiedenen Abschnitten des Jupyter-Notebooks zu vermeiden, wird hier eine Funktion definiert, die bei jedem Aufruf ein Dictionary mit den benötigten Widgets neu zur Verfügung stellt.

In [ ]:
def get_widgets():
    widget_dict = {
        "t_end":
        widgets.FloatSlider(
            value=100, min=10, max=10000, step=10,
            description=r"$t_\text{end}$"),
        "n_out":
        widgets.IntSlider(
            value=500, min=10, max=2000, step=10,
            description=r"$n_\text{out}$"),
        "atol":
        widgets.FloatLogSlider(
            value=1e-8, min=-10, max=-3, step=1,
            description="atol"),
        "rtol":
        widgets.FloatLogSlider(
            value=1e-8, min=-10, max=-3, step=1,
            description="rtol"),
        "scale":
        widgets.RadioButtons(
            options=(1, 100), value=100,
            description="Skalierung der Erddrehrate"),
        "algorithm":
        widgets.RadioButtons(
            options=["RK45", "RK23", "DOP853",
                     "Radau", "BDF", "LSODA"],
            value="RK45",
            description="Lösungsalgorithmus")
    }
    return widget_dict

### Implementierung der Differentialgleichungen

Neben einer Funktion, die die vier Differentialgleichungen 1. Ordnung implementiert, wird hier auch eine Funktion für die Jacobi-Matrix definiert, die von einigen der von `integrate.solve_ivp` unterstützten Löser benötigt wird.

In [ ]:
def dx_dt(t, x, omega_E):
    x1, v1, x2, v2 = x
    return v1, -x1 + 2*omega_E*v2, v2, -x2 - 2*omega_E*v1

In [ ]:
def jac(t, x, omega_E):
    return ([0, 1, 0, 0], [-1, 0, 0, 2*omega_E],
            [0, 0, 0, 1], [0, -2*omega_E, -1, 0])

### Lösung des Differentialgleichungssystems

Die Lösung des Differentialgleichungssystems erfolgt wieder mit Hilfe der SciPy-Funktion `integrate.solve_ivp`. Für bestimmte Löser wird zusätzlich zu den anderen Parametern die Funktion zur Berechnung der Jacobi-Matrix im Dictionary `kwargs` gesetzt. Neben den Ergebnissen für die Koordinaten der Projektion der Position der Pendelmasse in die $x$-$y$-Ebene wird auch die Anzahl der Aufrufe der Funktionen `dx_dt` und `jac` zurückgegeben, um die Effizienz des betreffenden Lösers beurteilen zu können.

In [ ]:
def trajectory(t_end, n_out, x0, atol, rtol, algorithm,
               omega_E):
    t_values = np.linspace(0, t_end, n_out)
    kwargs = {"t_eval": t_values, "args": (omega_E,),
              "atol": atol, "rtol": rtol,
              "method": algorithm, "dense_output": True}
    if algorithm in ("BDF", "LSODA", "Radau"):
        kwargs["jac"] = jac
    solution = integrate.solve_ivp(dx_dt, (0, t_end),
                                   x0, **kwargs)
    return (solution.y[0], solution.y[2],
            solution.nfev, solution.njev)

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit Hilfe der graphischen Benutzerschnittstelle lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `atol`: Schranke für den absoluten Fehler
- `rtol`: Schranke für den relativen Fehler
- `scale`: Skalierung der dimensionslosen Rotationsfrequenz der Erde bezogen auf das historische Pendel mit einer Länge von 67m und der geographischen Breite von Paris. Da diese Rotationsfrequenz sehr klein ist, gibt es die Möglichkeit einer Beschleunigung der Rotation um einen Faktor 100.
- `algorithm`: zu verwendender Löser

Innerhalb der Funktion `plot_trajectory` ist der Wert für die Rotationsfrequenz der Erde fest kodiert. Ferner ist die Anfangsbedingung festgelegt, wobei das in $x$-Richtung ausgelenkte Pendel aus der Ruhe losgelassen wird.

Bei der graphischen Darstellung der Bahnkurve wird mit Hilfe der Anweisung `ax.set_aspect("equal")` sichergestellt, dass keine Verzerrung auftritt. Ferner wird die Zahl der Aufrufe von `dx_dt` und von `jac` ausgegeben.

In [ ]:
@interact(**get_widgets())
def plot_trajectory(t_end, n_out, atol, rtol, scale,
                    algorithm):
    omega_E = 1.43e-4*scale
    x0 = [1, 0, 0, 0]
    x_values, y_values, count1, count2 = trajectory(
        t_end, n_out, x0, atol, rtol, algorithm, omega_E)

    print(f"{count1} Aufrufe von dx_dt")
    print(f"{count2} Aufrufe von jac")
    print("")
    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    ax.plot(x_values, y_values)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")

## Drehung der Schwingungsebene

### Festlegung der Zeitpunkte der Maximalauslenkungen

Die Richtung der Schwingung soll zu den Zeitpunkten berechnet werden, an denen die Maximalauslenkung erreicht ist. Zu diesen Zeitpunkten verschwindet die Radialgeschwindigkeit, so dass $x v_x + y v_y$ null sein muss. Um die Zeitpunkte auszuschließen, an denen die Minimalauslenkung erreicht wird, muss zusätzlich die Richtung `r_maximal.direction` auf -1 gessetzt werden.

In [ ]:
def r_maximal(t, x, *args):
    x1, v1, x2, v2 = x
    return x1*v1 + x2*v2

r_maximal.direction = -1

### Generierung der Lösung

Die Bewegungsgleichungen werden auch hier wieder mit Hilfe von `integrate.solve_ivp` gelöst, wobei der Löser explizit ausgewählt werden kann. Das Attribut `t_events` des Lösungsobjekts `solution` enthält die Zeitpunkte, an denen der maximale Abstand erreicht wird, und die zugehörigen Koordinaten lassen sich aus dem Attribut `y_events` entnehmen. Zur Bestimmung der Orientierung der Schwingungsebene wird hier die NumPy-Funktion `arctan2` verwendet, die  zwei separate Argumente verlangt und nicht deren Verhältnis, wie dies bei der NumPy-Funktion `arctan` der Fall ist. Damit können auch Fälle, in denen zum Beispiel das zweite Argument verschwindet, problemlos behandelt werden können.

Die zurückgegebenen Zeiten und Winkel werden in zwei Gruppen aufgeteilt, die zu jeweils gegenüberliegenden Umkehrpunkten gehören. Außerdem wird die Zahl der Aufrufe von `dx_dt` und `jac` übergeben.

In [ ]:
def orientation(t_end, n_out, x0, atol, rtol, algorithm,
                omega_E):
    t_values = np.linspace(0, t_end, n_out)
    kwargs = {"t_eval": t_values, "args": (omega_E,),
              "events": r_maximal,
              "atol": atol, "rtol": rtol,
              "method": algorithm, "dense_output": True}
    if algorithm in ("BDF", "LSODA", "Radau"):
        kwargs["jac"] = jac
    solution = integrate.solve_ivp(dx_dt, (0, t_end),
                                   x0, **kwargs)

    t_values = solution.t_events[0]
    phi_values = np.arctan2(solution.y_events[0][:, 2],
                            solution.y_events[0][:, 0])
    return (t_values[::2], phi_values[::2],
            t_values[1::2], phi_values[1::2],
            solution.nfev, solution.njev)

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit Hilfe der graphischen Benutzerschnittstelle lassen sich die folgenden Parameter einstellen:
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `atol`: Schranke für den absoluten Fehler
- `rtol`: Schranke für den relativen Fehler
- `scale`: Skalierung der dimensionslosen Rotationsfrequenz der Erde bezogen auf das historische Pendel mit einer Länge von 67m und der geographischen Breite von Paris. Da diese Rotationsfrequenz sehr klein ist, gibt es die Möglichkeit einer Beschleunigung der Rotation um einen Faktor 100.
- `algorithm`: zu verwendender Löser

Die Anfangsbedingungen werden in der Funktion `plot_angle` definiert und entsprechen den zuvor verwendeten Werten. Das zu betrachtende Zeitintervall umfasst einen gesamten Umlauf der Schwingungsebene.

Vor der graphischen Darstellung der zeitabhängigen Orientierung der Schwingungsebene wird noch die Zahl der Aufrufe von `dx_dt` und `jac` ausgegeben.

In [ ]:
interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**get_widgets())
def plot_angle(n_out, atol, rtol, scale, algorithm):
    omega_E = 1.43e-4*scale
    t_end = 2*pi/omega_E
    x0 = [1, 0, 0, 0]
    (t_1_values, phi_1_values, t_2_values, phi_2_values,
     count1, count2) = orientation(
        t_end, n_out, x0, atol, rtol, algorithm, omega_E)

    print(f"{count1} Aufrufe von dx_dt")
    print(f"{count2} Zahl der Aufrufe von jac")
    print("")
    fig, ax = plt.subplots()
    ax.plot(t_1_values, phi_1_values, linestyle="None",
            marker="o")
    ax.plot(t_2_values, phi_2_values, linestyle="None",
            marker="o")
    ax.set_xlabel(r"$\omega_\mathrm{E}\sin(\varphi)t$")
    ax.set_ylabel(r"$\phi$")

## Sphärisches Foucault-Pendel

### Implementierung der Differentialgleichungen

Die nachfolgende Funktion implementiert das nichtlineare  Differentialgleichungssystem. Dazu wandelt man den zweiten Satz an Bewegungsgleichungen, der in der Einleitung zu diesem Jupyter-Notebook angegeben ist, in vier Differentialgleichungen 1. Ordnung um.

In [ ]:
def dx_dt_spherical(t, y, omega_E, latitude):
    phi, theta, v_phi, v_theta = y
    a_theta = (0.5*v_phi**2*sin(2*theta)
               - sin(theta)
               - omega_E*v_phi * (
                   2*sin(theta)**2*sin(phi)*cos(latitude)
                   - sin(2*theta)*sin(latitude)))
    a_phi = (2*omega_E*v_theta*sin(phi)*cos(latitude)
             - 2*(v_phi+omega_E*sin(latitude))
                * v_theta/tan(theta))
    return v_phi, v_theta, a_phi, a_theta

### Lösung des Differentialgleichungssystems

Die Lösung des Differentialgleichungssystems erfolgt mit `integrate.solve_ivp` unter Verwendung des Defaultlösers.

Neben den Zeitabhängigkeiten der beiden Winkel und Winkelgeschwindigkeiten wird auch die Zahl der Aufrufe von `dx_dt_spherical` zurückgegeben.

In [ ]:
def solution(t_end, n_out, phi_0, theta_0, v_phi_0,
             v_theta_0, atol, rtol, omega_E, latitude):
    t_values = np.linspace(0, t_end, n_out)
    solution = integrate.solve_ivp(
        dx_dt_spherical, (0, t_end),
        [phi_0, theta_0, v_phi_0, v_theta_0],
        t_eval=t_values, atol=atol, rtol=rtol,
        args=(omega_E, latitude))
    phi, theta, v_phi, v_theta = solution.y[:]
    t = solution.t
    return t, phi, theta, v_phi, v_theta, solution.nfev

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit den Schiebereglern können die folgenden Parameter eingestellt werden:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `v_phi_0`: anfängliche Winkelgeschwindigkeit $\dot\phi$
- `theta_0`: Anfangswinkel $\theta$
- `v_theta_0`: anfängliche Winkelgeschwindigkeit $\dot\theta$
- `omega_E`: dimensionslose Rotationsfrequenz der Erde
- `latitude`: geographische Breite

Der anfängliche Winkel $\phi$ und die Fehlerschranken für die Integration werden in der Funktion `plot_trajectory_spherical` festgelegt und können dort bei Bedarf angepasst werden.

Die Ergebnisse werden als Projektion der Bahnkurve in die $x$-$y$-Ebene dargestellt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=130, min=10, max=300, step=10,
                   description=r"$t_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=500, min=10, max=2000, step=10,
                   description=r"$n_\text{out}$"),
               "v_phi_0":
               widgets.FloatSlider(
                   value=0.1, min=-1, max=1, step=0.1,
                   description=r"$v_\varphi(0)$"),
               "theta_0":
               widgets.FloatSlider(
                   value=pi/4, min=-3.14, max=3.14,
                   step=0.1, description=r"$\theta(0)$"),
               "v_theta_0":
               widgets.FloatSlider(
                   value=0, min=-2.5, max=2.5, step=0.1,
                   description=r"$v_\theta(0)$"),
               "omega_E":
               widgets.FloatSlider(
                   value=0.5, min=0.01, max=1, step=0.01,
                   description=r"$\omega_\text{E}$"),
               "latitude":
               widgets.FloatSlider(
                   value=pi/4, min=-pi/2, max=pi/2,
                   step=0.001, description=r"$\varphi$")
               }

@interact(**widget_dict)
def plot_trajectory_spherical(
        t_end, n_out, v_phi_0, theta_0, v_theta_0, omega_E,
        latitude):
    phi_0 = 0
    atol = 1e-10
    rtol = 1e-10
    t, phi, theta, v_phi, v_theta, count = solution(
        t_end, n_out, phi_0, theta_0, v_phi_0, v_theta_0,
        atol, rtol, omega_E, latitude)

    print(f"{count} Aufrufe von dx_dt_spherical")
    print("")
    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    ax.plot(np.cos(phi)*np.sin(theta),
            np.sin(phi)*np.sin(theta))
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")